## IndicTrans2

In [6]:
import torch
from transformers import AutoModelForSeq2SeqLM, BitsAndBytesConfig
# from IndicTransTokenizer import IndicProcessor, IndicTransTokenizer
from IndicTransToolkit import IndicProcessor
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from tqdm import tqdm
from torch.nn.functional import softmax


BATCH_SIZE = 16 # edited from 4
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
quantization = None
print(DEVICE)

cuda


In [70]:
def initialize_model_and_tokenizer(ckpt_dir, quantization):
    if quantization == "4-bit":
        qconfig = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_use_double_quant=True,
            bnb_4bit_compute_dtype=torch.bfloat16,
        )
    elif quantization == "8-bit":
        qconfig = BitsAndBytesConfig(
            load_in_8bit=True,
            bnb_8bit_use_double_quant=True,
            bnb_8bit_compute_dtype=torch.bfloat16,
        )
    else:
        qconfig = None

    tokenizer = AutoTokenizer.from_pretrained(ckpt_dir, trust_remote_code=True)
    model = AutoModelForSeq2SeqLM.from_pretrained(
        ckpt_dir,
        trust_remote_code=True,
        low_cpu_mem_usage=True,
        quantization_config=qconfig,
    )

    if qconfig == None:
        model = model.to(DEVICE)
        if DEVICE == "cuda":
            model.half()

    model.eval()

    return tokenizer, model


def batch_translate(input_sentences, src_lang, tgt_lang, model, tokenizer, ip):
    translations = []



    for i in tqdm(range(0, len(input_sentences), BATCH_SIZE)):

    # for i in range(0, len(input_sentences), BATCH_SIZE):
        batch = input_sentences[i : i + BATCH_SIZE]
        print("batch: ", batch)

        # Preprocess the batch and extract entity mappings
        batch = ip.preprocess_batch(batch, src_lang=src_lang, tgt_lang=tgt_lang)
        print("batch after pre-process: ", batch)

        # Tokenize the batch and generate input encodings
        inputs = tokenizer(
            batch,
            truncation=True,
            padding="longest",
            return_tensors="pt",
            return_attention_mask=True,
        ).to(DEVICE)
        print("inputs: ", inputs)

        # Generate translations using the model
        with torch.no_grad():
            # generated_tokens = model.generate(
            outputs = model.generate(
                **inputs,
                use_cache=True,
                min_length=0,
                max_length=256,
                num_beams=5,
                num_return_sequences=1, # TODO temp
                # output_scores=True,
                output_logits=True,
                return_dict_in_generate=True
            )
            
            # print("\n\ngenerated_tokens: ", dir(outputs))
            
            # print("\n\ngenerated_tokens: ", (outputs))
        
            # if generated_tokens.logits:
            #     print("\n\ngenerated_tokens LOGITS: ", (generated_tokens.logits))

            # Generated tokens
            
            # Scores (logits) for each token
            # scores = outputs.scores  # List of tensors, one for each generated token
            # print("1st score: ", scores[0].shape, len(scores))
            # Convert logits to probabilities
            # probabilities = [softmax(score, dim=-1) for score in scores]
            # print("\n\nprobabilities: ", probabilities)
            # print("0th probability: ", probabilities[0].shape)

            # print("output logits: ", outputs.logits)
            # size of logits: (batch_size, seq_len, vocab_size)
            print("len of output logits: ", len(outputs.logits))
            print("Shape of output logits [0]: ", outputs.logits[0].shape)
            # print("logit after 1st token: ", outputs.logits[0])

            # # Decode the generated tokens
            # generated_text = tokenizer.decode(generated_tokens[0], skip_special_tokens=True)

            # print("Generated Text:", generated_text)
            
            # print("\n\ngenerated_tokens LOGITS: ", (generated_tokens.logits()))

        # Decode the generated tokens into text
        generated_tokens = outputs.sequences
        print("len generated_tokens: ", (generated_tokens[0]).shape)
        vector = generated_tokens.detach().cpu().tolist()
        print("vector of generated_tokens: ", vector)


        with tokenizer.as_target_tokenizer():
            decoded_op = tokenizer.batch_decode(
                vector,
                # skip_special_tokens=True,
                clean_up_tokenization_spaces=True,
            )
        print("1st item in generated_tokens: ", decoded_op[0])
            
        print("generated_tokens after decode: ", decoded_op)
        for idx, (logit, token_id) in enumerate(zip(outputs.logits, vector[0])):
            # if idx!=0 and idx !=9:
            with tokenizer.as_target_tokenizer():

            
                token_str = tokenizer.batch_decode([token_id])
                # token_str = generated_tokens[0][idx]
            print(f"Step {idx + 1}: Token ID = {token_id}, Token = {token_str}, Logit Shape = {logit.shape}")
        # print decoded text and respective logit
        # for i in range(len(generated_tokens[0]).shape):
        #     print("Generated Text:", generated_tokens[0][i])
        #     print("Logit:", outputs.logits[i])
        #     print("\n\n")


        # Postprocess the translations, including entity replacement
        translations += ip.postprocess_batch(decoded_op, lang=tgt_lang)

        del inputs
        torch.cuda.empty_cache()

    return translations

**TODO: CAN I USE SOME CACHE TO STORE THE HF model and tokenizer?**

In [3]:
en_indic_ckpt_dir = "ai4bharat/indictrans2-en-indic-1B"  # ai4bharat/indictrans2-en-indic-dist-200M
en_indic_tokenizer, en_indic_model = initialize_model_and_tokenizer(en_indic_ckpt_dir,  quantization)

ip_en_ind = IndicProcessor(inference=True)

In [71]:
# import the test_sentences_eng.txt file data as sents

# sents = ['The river is blue.', 'My uncle wearing blue coat.', 'My maternal aunt went to the river bank.', "The doctor went to bank for money."]
sents=['My maternal aunt went to the river bank.']
src_lang = "eng_Latn"


tgt_lang = 'hin_Deva'
# print(lang)
translations = batch_translate(sents, src_lang, tgt_lang, en_indic_model, en_indic_tokenizer, ip_en_ind)


  0%|          | 0/1 [00:00<?, ?it/s]

batch:  ['My maternal aunt went to the river bank.']
batch after pre-process:  ['eng_Latn hin_Deva My maternal aunt went to the river bank .']
inputs:  {'input_ids': tensor([[    4,    15,   362, 14624, 17973,   436,     9,     6,  1811,   978,
             5,     2]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], device='cuda:0')}


/home/sofia/anaconda3/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:4126: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
100%|██████████| 1/1 [00:00<00:00,  2.32it/s]

len of output logits:  11
Shape of output logits [0]:  torch.Size([5, 122672])
len generated_tokens:  torch.Size([10])
vector of generated_tokens:  [[2, 1653, 8911, 3228, 9, 15933, 167, 3070, 6, 2]]
1st item in generated_tokens:  </s> मेरी नानी नदी के किनारे गई थीं ।</s> 
generated_tokens after decode:  ['</s> मेरी नानी नदी के किनारे गई थीं ।</s> ']
Step 1: Token ID = 2, Token = ['</s> '], Logit Shape = torch.Size([5, 122672])
Step 2: Token ID = 1653, Token = ['मेरी '], Logit Shape = torch.Size([5, 122672])
Step 3: Token ID = 8911, Token = ['नानी '], Logit Shape = torch.Size([5, 122672])
Step 4: Token ID = 3228, Token = ['नदी '], Logit Shape = torch.Size([5, 122672])
Step 5: Token ID = 9, Token = ['के '], Logit Shape = torch.Size([5, 122672])
Step 6: Token ID = 15933, Token = ['किनारे '], Logit Shape = torch.Size([5, 122672])
Step 7: Token ID = 167, Token = ['गई '], Logit Shape = torch.Size([5, 122672])
Step 8: Token ID = 3070, Token = ['थीं '], Logit Shape = torch.Size([5, 122672])
St